In [1]:
import xarray as xr
import s3fs
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
from datetime import datetime
import numpy as np

from dask.diagnostics import ProgressBar

# from dask.distributed import Client, LocalCluster

In [2]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))

## Dask with gateway

In [5]:
from dask.distributed import Client
from dask_gateway import Gateway
gateway = Gateway()

In [6]:
cluster = gateway.new_cluster(worker_memory=4, worker_cores=1)
cluster.scale(4)
client = Client(cluster) # create a dask Gateway cluster
cluster

In [7]:
if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /jupyterhub/services/dask-gateway/clusters/daskhub.9c4d9196fee44ed5a7fc0b9c130e71c1/status,


In [4]:
#closing open 
# clusters=gateway.list_clusters()
# print(clusters )
# for cluster in clusters :
#     cluster= gateway.connect(cluster.name)
#     print(cluster)
#     client = Client(cluster)
#     client.close()
#     cluster.shutdown()

In [6]:
# To close at end of session
client.close()

In [7]:
cluster.shutdown()

## List files

In [8]:
s3.ls('escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration')[-5:]

['escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111001-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111002-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111003-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111004-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/zip']

In [8]:
s3.ls('escience2022/Antoine/AMSR2_ASI_sea_ice_concentration')[-5:]

['escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20191227-v5.4.nc',
 'escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20191228-v5.4.nc',
 'escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20191229-v5.4.nc',
 'escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20191230-v5.4.nc',
 'escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20191231-v5.4.nc']

## Join files into yearly files

In [3]:
bucket = 's3://escience2022/Antoine/'


In [ ]:
with ProgressBar():
    for y in range(2002,2012):
        print('year '+ str(y))
        fE = bucket +'AMSR-E_ASI_sea_ice_concentration/asi-n6250-%i*.nc'%(y) 
        fileset = [s3.open(file) for file in s3.glob(fE)]
        dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
        # one file per day : read the time from the path
        time_var = xr.Variable('time', 
                [datetime.strptime(date.split('/')[-1].split('-')[2], '%Y%m%d') for date in s3.glob(fE)]              
                              )
        dset = dset.assign_coords(time=time_var)
        # saving
        sf = 'data/SIC/AMSR-E_ASI_sic_%i.nc'%(y)
        print('\t saving to '+ sf)
        dset.to_netcdf(sf)
        del dset, time_var


year 2002
	 saving to data/SIC/AMSR-E_ASI_sic_2002.nc
[######                                  ] | 17% Completed | 192.13 s

IOStream.flush timed out


[#######                                 ] | 19% Completed | 240.07 s

In [19]:
AMSR2files = [bucket +'AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-%i*.nc'%(y) for y in range(2012,2020)]

In [ ]:
## 